In [1]:
import pandas as pd

In [2]:
### Leer salida RF
df_RF = pd.read_csv('salidas/Salida_RF.csv',sep=';')
df_RF = df_RF[['Unnamed: 0', 'y', 'x','Codigo localidad', 'year', 'month',  'RF']]
df_RF

,Unnamed: 0,y,x,Codigo localidad,year,month,RF
0,0.0,10.0,8.0,12.0,2008.0,1.0,23.896667
1,1.0,10.0,8.0,12.0,2008.0,2.0,20.176250
2,2.0,10.0,8.0,12.0,2008.0,3.0,20.058583
3,3.0,10.0,8.0,12.0,2008.0,4.0,19.938667
4,4.0,10.0,8.0,12.0,2008.0,5.0,16.638167
...,...,...,...,...,...,...,...
35835,35835.0,19.0,9.0,5.0,2020.0,12.0,11.910667
35836,35836.0,19.0,9.0,5.0,2021.0,1.0,12.458333
35837,35837.0,19.0,9.0,5.0,2021.0,2.0,14.698750
35838,35838.0,19.0,9.0,5.0,2021.0,3.0,18.077583


In [3]:
### Leer salida XGBoost
df_XG = pd.read_csv('salidas/Salida_XGBoost.csv',sep=';')
df_XG = df_XG[['Unnamed: 0',  'XG']]
df_XG

,Unnamed: 0,XG
0,0.0,18.770964
1,1.0,20.086890
2,2.0,21.070350
3,3.0,17.773022
4,4.0,16.014570
...,...,...
35835,35835.0,12.457398
35836,35836.0,12.196022
35837,35837.0,14.324752
35838,35838.0,14.548713


In [4]:
### Cruzar y promediar
df_pm25 = pd.merge(df_RF,df_XG,on = 'Unnamed: 0')
df_pm25['pm25_pred'] = 0.5*(df_pm25.RF+df_pm25.XG)
df_pm25


,Unnamed: 0,y,x,Codigo localidad,year,month,RF,XG,pm25_pred
0,0.0,10.0,8.0,12.0,2008.0,1.0,23.896667,18.770964,21.333815
1,1.0,10.0,8.0,12.0,2008.0,2.0,20.176250,20.086890,20.131570
2,2.0,10.0,8.0,12.0,2008.0,3.0,20.058583,21.070350,20.564467
3,3.0,10.0,8.0,12.0,2008.0,4.0,19.938667,17.773022,18.855844
4,4.0,10.0,8.0,12.0,2008.0,5.0,16.638167,16.014570,16.326368
...,...,...,...,...,...,...,...,...,...
35835,35835.0,19.0,9.0,5.0,2020.0,12.0,11.910667,12.457398,12.184032
35836,35836.0,19.0,9.0,5.0,2021.0,1.0,12.458333,12.196022,12.327178
35837,35837.0,19.0,9.0,5.0,2021.0,2.0,14.698750,14.324752,14.511751
35838,35838.0,19.0,9.0,5.0,2021.0,3.0,18.077583,14.548713,16.313148


In [5]:
df_pm25.columns

Index(['Unnamed: 0', 'y', 'x', 'Codigo localidad', 'year', 'month', 'RF', 'XG',
       'pm25_pred'],
      dtype='object')

In [6]:
df_pred = df_pm25[['Codigo localidad','year', 'month', 'pm25_pred']]
df_pred

,Codigo localidad,year,month,pm25_pred
0,12.0,2008.0,1.0,21.333815
1,12.0,2008.0,2.0,20.131570
2,12.0,2008.0,3.0,20.564467
3,12.0,2008.0,4.0,18.855844
4,12.0,2008.0,5.0,16.326368
...,...,...,...,...
35835,5.0,2020.0,12.0,12.184032
35836,5.0,2021.0,1.0,12.327178
35837,5.0,2021.0,2.0,14.511751
35838,5.0,2021.0,3.0,16.313148


In [7]:
df_pm25_mensual = df_pred.groupby(['Codigo localidad','year', 'month']).mean('pm25_pred').reset_index()
df_pm25_mensual.to_excel('salidas/pm25_pred_mensual.xlsx',index=False)

In [8]:
df_pm25_anual = df_pred.groupby(['Codigo localidad','year']).mean('pm25_pred').reset_index()
df_pm25_anual.drop(columns='month',inplace =True)
df_pm25_anual.to_excel('salidas/pm25_pred_anual.xlsx',index=False)